In [ ]:
import requests
from datetime import datetime, timedelta
import pandas as pd
from dotenv import load_dotenv
import os
from tqdm import tqdm
import zipfile
from pathlib import Path
import glob
import jquantsapi
import inspect

In [ ]:
load_dotenv()
my_mail_address:str = os.environ['JQUANTS_EMAIL']
my_password: str = os.environ['JQUANTS_PASSWORD']
cli = jquantsapi.Client(mail_address=my_mail_address, password=my_password)

In [ ]:
now = datetime.now()
five_days_ago = now - timedelta(days=5)

In [ ]:
document_list_df = pd.read_excel('document_list.xlsx')

In [ ]:
all_data = []

for i, document_info in tqdm(document_list_df.iterrows(), total=len(document_list_df), desc='CSV読み込み中', unit='ファイル'):
    file_pattern = f"documents/{document_info['docID']}/XBRL_TO_CSV/jpcrp030000*.csv"
    files = glob.glob(file_pattern)
    csv_df = pd.read_csv(files[0], encoding='UTF-16-LE', sep='\t')

    data = {
        '書類番号': document_info['docID'],
        '証券コード': document_info['secCode'],
        '提出者名': document_info['filerName'],
    }

    elements = [
        {'name': '１株当たり当期純利益', 'element_id': '{taxonomy}_cor:BasicEarningsLossPerShare{ifrs}SummaryOfBusinessResults', 'context_id': 'CurrentYearDuration'},
        {'name': '１株当たり純資産額', 'element_id': '{taxonomy}_cor:NetAssetsPerShare{ifrs}SummaryOfBusinessResults', 'context_id': 'CurrentYearInstant'},
        {'name': '売上高', 'element_id': '{taxonomy}_cor:NetSales{ifrs}SummaryOfBusinessResults', 'context_id': 'CurrentYearDuration'},
        {'name': '前期売上高', 'element_id': '{taxonomy}_cor:NetSales{ifrs}SummaryOfBusinessResults', 'context_id': 'Prior1YearDuration'},
        {'name': '総資産', 'element_id': '{taxonomy}_cor:TotalAssets{ifrs}SummaryOfBusinessResults', 'context_id': 'CurrentYearInstant'},
        {'name': '純資産', 'element_id': '{taxonomy}_cor:NetAssets{ifrs}SummaryOfBusinessResults', 'context_id': 'CurrentYearInstant'},
        {'name': '現金及び預金', 'element_id': '{taxonomy}_cor:CashAndDeposits{ifrs}', 'context_id': 'CurrentYearInstant'},
        {'name': '土地', 'element_id': '{taxonomy}_cor:Land{ifrs}', 'context_id': 'CurrentYearInstant'},
        {'name': '投資有価証券', 'element_id': '{taxonomy}_cor:InvestmentSecurities{ifrs}', 'context_id': 'CurrentYearInstant'},
        {'name': '短期借入金', 'element_id': '{taxonomy}_cor:ShortTermLoansPayable{ifrs}', 'context_id': 'CurrentYearInstant'},
        {'name': '長期借入金', 'element_id': '{taxonomy}_cor:LongTermLoansPayable{ifrs}', 'context_id': 'CurrentYearInstant'},
        {'name': '営業キャッシュフロー', 'element_id': '{taxonomy}_cor:NetCashProvidedByUsedInOperatingActivities{ifrs}', 'context_id': 'CurrentYearDuration'},
        {'name': 'のれん', 'element_id': '{taxonomy}_cor:Goodwill{ifrs}', 'context_id': 'CurrentYearInstant'},
        {'name': '売掛金', 'element_id': '{taxonomy}_cor:AccountsReceivableTrade{ifrs}', 'context_id': 'CurrentYearInstant'},
        {'name': '前期売掛金', 'element_id': '{taxonomy}_cor:AccountsReceivableTrade{ifrs}', 'context_id': 'Prior1YearInstant'},
    ]

    for element in elements:
        found = False
        for non_consolidated in ['', '_NonConsolidatedMember']:
            for ifrs in ['', 'IFRS']:
                for taxonomy in ['jpcrp', 'jppfs', 'jpigp']:
                    result = csv_df[(csv_df['要素ID'] == element['element_id'].replace('{taxonomy}', taxonomy).replace('{ifrs}', ifrs)) & (csv_df['コンテキストID'] == element['context_id'] + non_consolidated)]
                    for value in result['値']:
                        numerical_value = pd.to_numeric(value, errors='coerce')
                        if pd.notna(numerical_value):
                            data[element['name']] = numerical_value
                            found = True
                        if found:
                            break
                    if found:
                        break
                if found:
                    break
            if found:
                break

    all_data.append(data)

    # break
    # if i == 99:
    #     break

df = pd.DataFrame(all_data)

In [ ]:
for i, row in tqdm(df.iterrows(), total=len(df)):
    quotes_df = cli.get_prices_daily_quotes(code=row['証券コード'], from_yyyymmdd=five_days_ago.strftime('%Y%m%d'))
    if len(quotes_df):
        last_quotes = quotes_df.iloc[-1]
        df.loc[i, '株価'] = last_quotes['Close']

In [ ]:
df['自己資本比率'] = df['純資産'] / df['総資産']
df['PER'] = df['株価'] / df['１株当たり当期純利益']
df['PBR'] = df['株価'] / df['１株当たり純資産額']
df['ネットキャッシュ'] = df['現金及び預金'].fillna(0) + df['投資有価証券'].fillna(0) - df['短期借入金'].fillna(0) - df['長期借入金'].fillna(0)
df['赤字額'] = (-df['営業キャッシュフロー']).clip(lower=0)
df['枯渇年数'] = df['ネットキャッシュ'] / df['赤字額']
df['売上高前年比'] = (df['売上高'] - df['前期売上高']) / df['前期売上高']
df['売掛金前年比'] = (df['売掛金'] - df['前期売掛金']) / df['前期売掛金']

In [ ]:
df.to_excel('basic_data.xlsx', index=False)